# Website Instructions

## To rebuild site and deploy

Run this and it will convert blog posts to html and push to google cloud.  It may take a few hours for the changes to reflect on the website.

## To add a blog post

+ The notebook must be named in this format `yyyy-mm-dd-title`
+ The first cell must be markdown and formatted in a particular way with particular information.  Look at the first cell in an existing blog post as reference.
+ Blog posts are jupyter notebooks stored in `website/posts/notebooks/`.  Move any notebooks there - this notebook will pick them up and do the conversion to html for you.  For example: `cp WIP/example.ipynb website/posts/notebooks/2021-12-31-blogtitle.ipynb`

In [52]:
import os
from fastcore.foundation import *
from pathlib import Path

In [53]:

header = '''
  <header id="header" class="header">
    <!--Logo-->
      <a class="logo" href="index.html"></a>
     <!--Navbar-->
     <nav class="navbar">
       <ul class="menu-items">
              <li><a class="menu-item" href="../../index.html">HOME</a></li>
              <li><a class="menu-item" href="../../assets.html">DATA ASSETS</a></li>
             <li><a class="menu-item" href="../../blog.html">BLOG</a></li>
           </nav>
         </li>
       </header>
       <br><br><br><br><br><br>
'''

css = '''


  .header {
    background:rgba(3, 3, 3, .5);
    box-shadow: 0 8px 32px 0 rgba(31,38,135,0.37);
    backdrop-filter: blur(5px);
    -webkit-backdrop-filter: blur(5px);
    border: 1px solid rgba(255,255,255,0.18);
    position: fixed;
    width: 90vw;
    margin: auto;
    display: flex;
    align-items: center;
    justify-content: space-between;
    padding: 1rem 1.8rem;
    z-index: 1;
  }

  .header a {
    text-decoration: none;
    color: #EAEAEA;
    font-size: calc(0.85rem + 1vw);
    font-weight: 600;
    letter-spacing: 0.1rem;
  }

  .menu-items {
    display: flex;
    align-items: center;
  }
.menu-items li {
  list-style-type: none;
}
  .menu-items li a {
    padding: 0.5rem 1rem;
    transition: background 0.3s ease-in-out;
    position: relative;
  }

/* Hover effect main nav buttons */
  .menu-items li a:link::before,
  .menu-items li a:visited::before,
  .menu-items li a:link::after,
  .menu-items li a:visited::after {
  content: "";
  position: absolute;
  left: 0;
  width: 95%;
  height: 0.3rem;
  background-color: #FF2E63;
  transform: scaleX(0);
  transition: transform 0.6s cubic-bezier(1, 0, 0, 1);
  }

  .menu-items li a:link::before,
  .menu-items li a:visited::before {
    bottom: 0;
    transform-origin: left;
  }

  .menu-items li a:link::after,
  .menu-items li a:visited::after {
    bottom: 0;
    transform-origin: left;
  }

  .menu-items li a:hover::before,
  .menu-items li a:active::before,
  .menu-items li a:hover::after,
  .menu-items li a:active::after {
    transform: scaleX(1);
  }

  span.button {
    color: #FF2E63;
  }'''

In [54]:
posts_path = Path('./website/posts/')
html_path = posts_path/'html'
posts_path,html_path

(PosixPath('website/posts'), PosixPath('website/posts/html'))

In [55]:
def nb_to_post(input_path=posts_path/'notebooks',output_path=html_path):
    os.system(f"rm ./{output_path}/*.html")
    os.system(f'jupyter nbconvert --to html {input_path}/*.ipynb') # eventually --template basic then css style
    os.system(f"mv ./{input_path}/*.html ./{output_path}/")
    

nb_to_post()

[NbConvertApp] Converting notebook website/posts/notebooks/2021-09-25-527 IRS Filings Data.ipynb to html
[NbConvertApp] Writing 633492 bytes to website/posts/notebooks/2021-09-25-527 IRS Filings Data.html


In [56]:
def inject_header(html_path=html_path):
    html_files = [o for o in os.listdir(html_path) if o.endswith('.html')]
    for html_file in html_files:    
        print(html_file)
        with open(f"./{html_path}/{html_file}",'r') as f: html = f.readlines()
        css_done=False
        for i,line in enumerate(html):
            if line.startswith("<body"): html[i] = line + header
            if line.startswith('<style type="text/css">') and (css_done==False): 
                css_done = True
                html[i] = line + css

        with open(f"./{html_path}/{html_file}",'w') as f: f.write('\n'.join(html))

    
inject_header()

2021-09-25-527 IRS Filings Data.html


In [57]:
posts = L(os.listdir(html_path))
posts.sort(reverse=True)     

In [58]:
for post in [o for o in posts if o[-5:]=='.html']:

    with open(html_path/post) as file: htmlfile = file.read()

    section1 = htmlfile[:htmlfile.find("</body>")]
    section2 = htmlfile[htmlfile.find("</body>"):]
    new_section = '''\n\n<script src="postformatting.js"></script>\n\n'''

    new_html = section1+new_section+section2
    
    with open(html_path/post,'w') as file: file.write(new_html)

In [59]:
def get_slice(file,start_str,end_str):
    with open(file) as f:
        tmp = f.read()    
    slice1 = tmp[tmp.find(start_str)+len(start_str):]
    out = slice1[:slice1.find(end_str)]
    return out

In [60]:
htmls = []
for post in posts:
    if post[-5:] != '.html': continue
    date = post[:10]
    
    name_tmp = post[11:-5]
    name = ''
    for i, letter in enumerate(name_tmp):
        if i and letter.isupper() and not name_tmp[i-1].isupper():
            name += ' '
        
        name += letter  
    description = get_slice(html_path/post,'Post Description:</strong> ','</p>')
    tags = get_slice(html_path/post,'Post Categories:</strong> ','</p>')

    htmls.append('<div class="smallblogdiv">')
    htmls.append(f'''<a class="postlink" href="./posts/html/{post}">{name}</a>''')
    htmls.append(f'''<p class="postdescription">{description}</p>''')
    htmls.append(f'''<p class="posttags">Tags: {tags}</p>''')
    htmls.append(f'''<p class="postdate">{date}</p>''')
    htmls.append('''</div>''')
htmls = '\n              '.join(htmls)

In [61]:
with open(posts_path/"blog_template.html") as f: tmp = f.read()
tmp = tmp.replace('AUTOINSERTIONPOINT',htmls)

with open(Path("website/blog.html"),'w') as f: f.write(tmp)

In [62]:
!gsutil -m rsync -d -r website/ gs://vinculum.solutions/

Building synchronization state...
If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Starting synchronization...
If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying file://website/blog.html [Content-Type=text/html]...
Removing gs://vinculum.solutions/posts/notebooks/.ipynb_checkpoints/2021-09-25-527 IRS Filings Data-checkpoint.ipynb
Copying file://website/index.js [Content-Type=application/